Reading the data   

In [70]:
import pyarrow.parquet as pa
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

driver_order = pa.read_table('data/masked_driver_order_mapping.parquet') 
driver_order = driver_order.to_pandas() 

order_articles = pa.read_table('data/masked_order_articles.parquet') 
order_articles = order_articles.to_pandas() 

orders = pa.read_table('data/masked_orders.parquet') 
orders = orders.to_pandas() 

service_time = pa.read_table('data/masked_service_times.parquet') 
service_time = service_time.to_pandas() 

# Taking tanspose so the printing dataset will easy. 
driver_order

,web_order_id,driver_id
0,51222,182
1,53503,182
2,166709,182
3,136190,182
4,176394,182
...,...,...
1534811,473982,9240
1534812,474113,9240
1534813,475628,9240
1534814,468676,9240


In [3]:
print(order_articles.describe())
order_articles


       warehouse_id        box_id    article_id  article_weight_in_g  \
count  1.558638e+07  8.091248e+06  1.558638e+07         1.558638e+07   
mean   2.237072e+01  2.451816e+07  1.024022e+04         7.909714e+03   
std    1.199967e+01  3.752282e+05  7.476332e+03         7.916583e+03   
min    1.000000e+00  0.000000e+00  6.000000e+00         6.000000e+00   
25%    1.200000e+01  2.419974e+07  3.541000e+03         3.660000e+02   
50%    2.300000e+01  2.451777e+07  8.484000e+03         1.353000e+03   
75%    3.200000e+01  2.484370e+07  1.769600e+04         1.695000e+04   
max    4.600000e+01  2.518101e+07  2.370500e+04         2.260000e+04   

       web_order_id  
count  1.558638e+07  
mean   7.653445e+05  
std    4.441621e+05  
min    1.000000e+00  
25%    3.822260e+05  
50%    7.570010e+05  
75%    1.155535e+06  
max    1.534834e+06  


,warehouse_id,box_id,article_id,article_weight_in_g,web_order_id
0,3,NaN,3657,17250,1250399
1,3,NaN,3657,17250,1250154
2,32,NaN,2576,17000,1296951
3,32,NaN,3975,15100,978668
4,25,NaN,3670,11000,104995
...,...,...,...,...,...
15586377,35,NaN,3953,17800,1533370
15586378,35,NaN,3953,17800,1533370
15586379,35,NaN,3953,17800,1533370
15586380,35,NaN,3953,17800,1533370


In [4]:
print(order_articles['box_id'].unique())

[      nan        0. 24510740. ... 24510737. 24510738. 24510739.]


In [5]:
orders


,warehouse_id,order_time,has_elevator,floor,is_pre_order,is_business,web_order_id,customer_id
0,18,2024-12-17 09:00:00+01:00,False,0.0,True,True,806432,166859
1,8,2024-12-10 09:45:00+01:00,False,0.0,True,True,678738,167463
2,8,2024-11-21 10:45:00+01:00,False,NaN,True,True,347665,49336
3,13,2024-11-08 10:00:00+01:00,False,0.0,True,True,110643,172552
4,13,2025-01-06 13:15:48.730000+01:00,False,NaN,False,True,1139043,136281
...,...,...,...,...,...,...,...,...
1534684,4,2024-11-27 15:45:00+01:00,False,0.0,True,True,460582,433427
1534685,6,2025-01-20 09:11:40.430000+01:00,False,0.0,False,True,1382719,404857
1534686,6,2024-11-26 08:45:00+01:00,False,0.0,True,True,419104,404857
1534687,6,2024-12-02 13:15:00+01:00,False,0.0,True,True,542528,393199


In [6]:
service_time

,service_time_start,service_time_end,service_time_in_minutes,order_datetime,web_order_id,driver_id,trip_id,customer_id
0,2024-11-11 21:48:59+01:00,2024-11-11 21:57:56+01:00,8.950000,2024-11-11 20:45:00+01:00,1,3621,29687,35422
1,2024-11-05 13:27:30+01:00,2024-11-05 13:45:05+01:00,17.583333,2024-11-05 13:15:00+01:00,2,6575,10097,159165
2,2024-11-05 21:52:03+01:00,2024-11-05 22:09:28+01:00,17.416666,2024-11-05 20:00:00+01:00,3,6811,12316,377355
3,2024-11-12 21:01:30.013000+01:00,2024-11-12 21:15:30.087000+01:00,14.000000,2024-11-12 19:00:00+01:00,4,5183,31795,146455
4,2024-11-01 15:28:07+01:00,2024-11-01 15:37:01+01:00,8.900000,2024-11-01 13:45:00+01:00,5,694,966,210296
...,...,...,...,...,...,...,...,...
1534829,2025-01-30 09:28:42+01:00,2025-01-30 09:33:43+01:00,5.016666,2025-01-30 08:10:51.200000+01:00,1534830,10536,250427,454002
1534830,2025-01-30 09:36:48+01:00,2025-01-30 09:45:29+01:00,8.683333,2025-01-30 08:13:38.747000+01:00,1534831,10755,250425,221139
1534831,2025-01-30 09:49:06+01:00,2025-01-30 09:58:29+01:00,9.383333,2025-01-30 08:15:32.833000+01:00,1534832,6570,250428,529887
1534832,2025-01-30 09:59:50+01:00,2025-01-30 10:07:27+01:00,7.616666,2025-01-30 08:47:31.837000+01:00,1534833,10925,250431,365722


joining the tables


In [7]:
# merged_data = pd.concat([orders, order_articles], axis=1, join="inner")
result1 = pd.merge(orders, order_articles, on=["web_order_id"])
result2 = pd.merge(result1, service_time, on=["web_order_id"])
result2

,warehouse_id_x,order_time,has_elevator,floor,is_pre_order,is_business,web_order_id,customer_id_x,warehouse_id_y,box_id,article_id,article_weight_in_g,service_time_start,service_time_end,service_time_in_minutes,order_datetime,driver_id,trip_id,customer_id_y
0,18,2024-12-17 09:00:00+01:00,False,0.0,True,True,806432,166859,18,NaN,2344,15050,2024-12-17 10:09:30.067000+01:00,2024-12-17 10:27:00.030000+01:00,17.500000,2024-12-17 09:00:00+01:00,9045,133625,166859
1,18,2024-12-17 09:00:00+01:00,False,0.0,True,True,806432,166859,18,NaN,2346,15050,2024-12-17 10:09:30.067000+01:00,2024-12-17 10:27:00.030000+01:00,17.500000,2024-12-17 09:00:00+01:00,9045,133625,166859
2,18,2024-12-17 09:00:00+01:00,False,0.0,True,True,806432,166859,18,NaN,1988,13900,2024-12-17 10:09:30.067000+01:00,2024-12-17 10:27:00.030000+01:00,17.500000,2024-12-17 09:00:00+01:00,9045,133625,166859
3,18,2024-12-17 09:00:00+01:00,False,0.0,True,True,806432,166859,18,NaN,2574,17250,2024-12-17 10:09:30.067000+01:00,2024-12-17 10:27:00.030000+01:00,17.500000,2024-12-17 09:00:00+01:00,9045,133625,166859
4,18,2024-12-17 09:00:00+01:00,False,0.0,True,True,806432,166859,18,NaN,2574,17250,2024-12-17 10:09:30.067000+01:00,2024-12-17 10:27:00.030000+01:00,17.500000,2024-12-17 09:00:00+01:00,9045,133625,166859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15585011,37,2024-12-06 11:15:00+01:00,False,0.0,True,False,608605,380052,37,NaN,1817,20600,2024-12-06 13:10:39+01:00,2024-12-06 13:23:55+01:00,13.266666,2024-12-06 11:15:00+01:00,9149,100124,380052
15585012,37,2024-12-06 11:15:00+01:00,False,0.0,True,False,608605,380052,37,24395771.0,3593,1350,2024-12-06 13:10:39+01:00,2024-12-06 13:23:55+01:00,13.266666,2024-12-06 11:15:00+01:00,9149,100124,380052
15585013,37,2024-12-06 11:15:00+01:00,False,0.0,True,False,608605,380052,37,24395771.0,3593,1350,2024-12-06 13:10:39+01:00,2024-12-06 13:23:55+01:00,13.266666,2024-12-06 11:15:00+01:00,9149,100124,380052
15585014,37,2024-12-06 11:15:00+01:00,False,0.0,True,False,608605,380052,37,24395771.0,3593,1350,2024-12-06 13:10:39+01:00,2024-12-06 13:23:55+01:00,13.266666,2024-12-06 11:15:00+01:00,9149,100124,380052


Dropping columns


In [8]:
result3 = result2.drop(['order_time', 'customer_id_x', 'warehouse_id_y', 'service_time_start', 'service_time_end', 'order_datetime', 'customer_id_y', 'trip_id'], axis=1)
result3

,warehouse_id_x,has_elevator,floor,is_pre_order,is_business,web_order_id,box_id,article_id,article_weight_in_g,service_time_in_minutes,driver_id
0,18,False,0.0,True,True,806432,NaN,2344,15050,17.500000,9045
1,18,False,0.0,True,True,806432,NaN,2346,15050,17.500000,9045
2,18,False,0.0,True,True,806432,NaN,1988,13900,17.500000,9045
3,18,False,0.0,True,True,806432,NaN,2574,17250,17.500000,9045
4,18,False,0.0,True,True,806432,NaN,2574,17250,17.500000,9045
...,...,...,...,...,...,...,...,...,...,...,...
15585011,37,False,0.0,True,False,608605,NaN,1817,20600,13.266666,9149
15585012,37,False,0.0,True,False,608605,24395771.0,3593,1350,13.266666,9149
15585013,37,False,0.0,True,False,608605,24395771.0,3593,1350,13.266666,9149
15585014,37,False,0.0,True,False,608605,24395771.0,3593,1350,13.266666,9149


In [9]:
# Group by 'web_order_id' and count the number of 'box_id' 
# COUNT UNIQUE BOX IDS
num_boxes = result3.groupby('web_order_id')['box_id'].nunique().reset_index()

# Rename the column for clarity
num_boxes.rename(columns={'box_id': 'box_count'}, inplace=True)

print(num_boxes)

         web_order_id  box_count
0                   1          2
1                   2          0
2                   3          0
3                   4          4
4                   5          0
...               ...        ...
1534647       1534830          0
1534648       1534831          0
1534649       1534832          0
1534650       1534833          1
1534651       1534834          2

[1534652 rows x 2 columns]


In [66]:
result4 = pd.merge(result3, num_boxes, on=["web_order_id"])
result4 = result4.drop(['box_id'], axis=1)
result4

,warehouse_id_x,has_elevator,floor,is_pre_order,is_business,web_order_id,article_id,article_weight_in_g,service_time_in_minutes,driver_id,box_count
0,18,False,0.0,True,True,806432,2344,15050,17.500000,9045,0
1,18,False,0.0,True,True,806432,2346,15050,17.500000,9045,0
2,18,False,0.0,True,True,806432,1988,13900,17.500000,9045,0
3,18,False,0.0,True,True,806432,2574,17250,17.500000,9045,0
4,18,False,0.0,True,True,806432,2574,17250,17.500000,9045,0
...,...,...,...,...,...,...,...,...,...,...,...
15585011,37,False,0.0,True,False,608605,1817,20600,13.266666,9149,1
15585012,37,False,0.0,True,False,608605,3593,1350,13.266666,9149,1
15585013,37,False,0.0,True,False,608605,3593,1350,13.266666,9149,1
15585014,37,False,0.0,True,False,608605,3593,1350,13.266666,9149,1


The problem I have is that in one order there are multiple articles. However, in total there are thousands of articles so I can't one-hot-encode. I dont want to have multiple rows that are the exact same but just differ in 2 columns, article_id and its weight.

## First approach will be to try and aggregrate the data. So number of articles and summing the weight. Second Approach is One-Hot-encoding whereby we will devide this into 2 subcategories. First, the value of the encoding will be 0 or 1 and second, the values will be 0 or the num of elements in the order.

# Important idea: We want to extract box_count, crate_count and the weights of the boxed (box_id_weight)


In [67]:
# Aggregate articles as lists, sum weights, and count total articles
result5 = result4.groupby('web_order_id').agg({
    'article_id': [list, 'count'],  # Store articles in a list and count total
    'article_weight_in_g': 'sum'  # Sum of article weights
}).reset_index()

# Rename columns for clarity
result5.columns = ['web_order_id', 'articles', 'total_articles', 'total_weight']



In [68]:
result5 = result5[['web_order_id','total_articles', 'total_weight']]
result5


,web_order_id,total_articles,total_weight
0,1,17,20443
1,2,4,56400
2,3,11,189750
3,4,24,54191
4,5,7,79590
...,...,...,...
1534647,1534830,6,88800
1534648,1534831,4,71200
1534649,1534832,10,153502
1534650,1534833,3,33738


In [69]:
result6 = pd.merge(result5, result4 , on=["web_order_id"])
result6 = result6.drop(['article_weight_in_g'], axis=1)
result6

,web_order_id,total_articles,total_weight,warehouse_id_x,has_elevator,floor,is_pre_order,is_business,article_id,service_time_in_minutes,driver_id,box_count
0,1,17,20443,27,True,7.0,True,False,1302,8.950000,3621,2
1,1,17,20443,27,True,7.0,True,False,9922,8.950000,3621,2
2,1,17,20443,27,True,7.0,True,False,19488,8.950000,3621,2
3,1,17,20443,27,True,7.0,True,False,22954,8.950000,3621,2
4,1,17,20443,27,True,7.0,True,False,22312,8.950000,3621,2
...,...,...,...,...,...,...,...,...,...,...,...,...
15585011,1534834,16,34769,17,True,5.0,False,False,21371,10.866666,10925,2
15585012,1534834,16,34769,17,True,5.0,False,False,21374,10.866666,10925,2
15585013,1534834,16,34769,17,True,5.0,False,False,21372,10.866666,10925,2
15585014,1534834,16,34769,17,True,5.0,False,False,18046,10.866666,10925,2


# Training and test split

In [61]:
result7 = result6.drop('article_id', axis=1)
result7 = result7.drop_duplicates(subset=['web_order_id'])
result7['floor'] = result7['floor'].fillna(0)
# result7['driver_id'] = result7['driver_id'].astype(str)


X = result7.drop('service_time_in_minutes', axis=1)
y = result7[['service_time_in_minutes']]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)


result7


,web_order_id,total_articles,total_weight,has_elevator,floor,is_pre_order,is_business,service_time_in_minutes,driver_id,box_count
0,1,17,20443,True,7.0,True,False,8.950000,3621,2
17,2,4,56400,False,2.0,True,False,17.583333,6575,0
21,3,11,189750,False,1.0,True,False,17.416666,6811,0
32,4,24,54191,False,2.0,True,False,14.000000,5183,4
56,5,7,79590,True,0.0,True,False,8.900000,694,0
...,...,...,...,...,...,...,...,...,...,...
15584977,1534830,6,88800,False,0.0,False,False,5.016666,10536,0
15584983,1534831,4,71200,False,3.0,False,False,8.683333,10755,0
15584987,1534832,10,153502,True,-1.0,False,True,9.383333,6570,0
15584997,1534833,3,33738,True,1.0,False,False,7.616666,10925,1


In [62]:
print(result7.isna().sum())
print(result7.dtypes)



web_order_id               0
total_articles             0
total_weight               0
has_elevator               0
floor                      0
is_pre_order               0
is_business                0
service_time_in_minutes    0
driver_id                  0
box_count                  0
dtype: int64
web_order_id                 int64
total_articles               int64
total_weight                 int32
has_elevator                  bool
floor                      float64
is_pre_order                  bool
is_business                   bool
service_time_in_minutes    float64
driver_id                    int64
box_count                    int64
dtype: object


# Training the Random Forest

In [63]:
regressor = RandomForestRegressor(n_estimators=20, random_state=0, oob_score=True)

regressor.fit(X_train, y_train)

/opt/anaconda3/envs/web_mining/lib/python3.9/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/web_mining/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:612: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestRegressor(n_estimators=20, oob_score=True, random_state=0)

In [64]:
oob_score = regressor.oob_score_
print(f'Out-of-Bag Score: {oob_score}')

predictions = regressor.predict(X_train)

mse = mean_squared_error(y_train, predictions)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_train, predictions)
print(f'R-squared: {r2}')

# Old array fit
# Out-of-Bag Score: 0.6799009762762025
# Mean Squared Error: 1.249703034044174
# R-squared: 0.9782934136908276

Out-of-Bag Score: 0.2520486718472258
Mean Squared Error: 4.146378544761159
R-squared: 0.8928090758018253


In [65]:
oob_score = regressor.oob_score_
print(f'Out-of-Bag Score: {oob_score}')

predictions = regressor.predict(X_test)

mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, predictions)
print(f'R-squared: {r2}')

# Old array fit
# Out-of-Bag Score: 0.6799009762762025
# Mean Squared Error: 1.249703034044174
# R-squared: 0.9782934136908276

Out-of-Bag Score: 0.2520486718472258
Mean Squared Error: 26.734523993502098
R-squared: 0.31453178982623853


# XGBoost


In [71]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

param['eval_metric'] = ['auc', 'ams@0']

evallist = [(dtrain, 'train'), (dtest, 'eval')]

num_round = 10
bst = xgb.train(param, dtrain, num_round, evallist)

ypred = bst.predict(dtest)

/opt/anaconda3/envs/web_mining/lib/python3.9/site-packages/xgboost/core.py:723: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


XGBoostError: [17:55:50] /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/objective/regression_obj.cu:88: label must be in [0,1] for logistic regression
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000003aceef1f0 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x00000003ad18b10c xgboost::obj::RegLossObj<xgboost::obj::LogisticClassification>::ValidateLabel(xgboost::MetaInfo const&) + 140
  [bt] (2) 3   libxgboost.dylib                    0x00000003ad18ae1c xgboost::obj::RegLossObj<xgboost::obj::LogisticClassification>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::linalg::Tensor<xgboost::detail::GradientPairInternal<float>, 2>*) + 96
  [bt] (3) 4   libxgboost.dylib                    0x00000003ad15549c xgboost::obj::FitIntercept::InitEstimation(xgboost::MetaInfo const&, xgboost::linalg::Tensor<float, 1>*) const